In [1]:
# To run this in Google Colab, uncomment the following line
# !pip install geometric_kernels

# If you want to use a version of the library from a specific branch on GitHub,
# say, from the "devel" branch, uncomment the line below instead
# !pip install "git+https://github.com/geometric-kernels/GeometricKernels@devel"

# Benchmarking Speed Improvements on Hypercube/Hamming Graphs

This notebook benchmarks two methods for the `HypercubeGraph` and `HammingGraph` spaces with $\nu = \infty$:

1. "slow" method — based on the addition theorem presented on this [documentation page](https://geometric-kernels.github.io/GeometricKernels/theory/hamming_graph.html),
2. "fast" method — based on the closed-form equations discusssed in [Kondor and Lafferty (2002)](https://people.cs.uchicago.edu/~risi/papers/diffusion-kernels.pdf) or [Doumont et al. (2025)](https://arxiv.org/pdf/2510.26633).

Moreover, we also double-check whether the different methods lead to equal kernels.


In [2]:
import time
from collections import defaultdict

import numpy as np

from geometric_kernels.kernels import MaternKernelHammingGraph
from geometric_kernels.kernels.karhunen_loeve import MaternKarhunenLoeveKernel
from geometric_kernels.spaces import HammingGraph, HypercubeGraph

INFO (geometric_kernels): Numpy backend is enabled. To enable other backends, don't forget to `import geometric_kernels.*backend name*`.
INFO (geometric_kernels): We may be suppressing some logging of external libraries. To override the logging policy, call `logging.basicConfig`.
/opt/miniconda3/envs/geometrik/lib/python3.11/site-packages/spherical_harmonics/fundamental_set.py:21: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [3]:
SEED = 1234
np.random.seed(SEED)

# Configuration
SPACE_DIM = 20  # dimensionality of the space
N_CAT = 5  # number of categories (alphabet size) for Hamming graph
N_SAMPLES = 3000  # number of points to evalute the kernels on
N_RUNS = 10  # number of seeds to average over
LENGTHSCALES = [1.0, 5.0, 10.0, 20.0]

In [4]:
def generate_data(space, n, seed_offset=0):
    key1 = np.random.RandomState(1234 + seed_offset)
    key2 = np.random.RandomState(5678 + seed_offset)
    _, X1 = space.random(key1, n)
    _, X2 = space.random(key2, n)
    return X1, X2


def run_benchmark(space, kernel_type, lengthscales, n_samples, n_runs, space_dim):
    results = {"times": defaultdict(list), "kernels": {}}

    for lengthscale in lengthscales:
        for run_idx in range(n_runs):
            if kernel_type == "fast":
                kernel = MaternKernelHammingGraph(space, num_levels=space_dim + 1)
            else:
                kernel = MaternKarhunenLoeveKernel(space, num_levels=space_dim + 1)

            X1, X2 = generate_data(space, n_samples, seed_offset=run_idx * 1)
            params = dict(nu=np.array([np.inf]), lengthscale=np.array([lengthscale]))

            start = time.perf_counter()
            K = kernel.K(params, X1, X2)
            elapsed = time.perf_counter() - start

            results["times"][lengthscale].append(elapsed)
            if run_idx == 0:
                results["kernels"][lengthscale] = K

    return results


def print_results(fast_results, slow_results, lengthscales, space_name):
    print(f"\n{'='*80}\n{space_name}\n{'='*80}")

    for ls in lengthscales:
        fast_mean = np.mean(fast_results["times"][ls])
        slow_mean = np.mean(slow_results["times"][ls])
        speedup = slow_mean / fast_mean
        is_equal = np.allclose(fast_results["kernels"][ls], slow_results["kernels"][ls])

        print(
            f"Lengthscale {ls:5.1f}: Fast={fast_mean:.4f}s | Slow={slow_mean:.4f}s | "
            f"Speedup={speedup:.2f}x | Equal={is_equal}"
        )

In [5]:
print("Running...")
print("")

# HypercubeGraph
graph = HypercubeGraph(SPACE_DIM)
fast_results = run_benchmark(graph, "fast", LENGTHSCALES, N_SAMPLES, N_RUNS, SPACE_DIM)
slow_results = run_benchmark(graph, "slow", LENGTHSCALES, N_SAMPLES, N_RUNS, SPACE_DIM)
print_results(fast_results, slow_results, LENGTHSCALES, "HYPERCUBE GRAPH")

# HammingGraph
hamming_graph = HammingGraph(SPACE_DIM, N_CAT)
fast_results = run_benchmark(
    hamming_graph, "fast", LENGTHSCALES, N_SAMPLES, N_RUNS, SPACE_DIM
)
slow_results = run_benchmark(
    hamming_graph, "slow", LENGTHSCALES, N_SAMPLES, N_RUNS, SPACE_DIM
)
print_results(fast_results, slow_results, LENGTHSCALES, "HAMMING GRAPH")

print("")
print("Done!")

Running...


HYPERCUBE GRAPH
Lengthscale   1.0: Fast=0.6992s | Slow=1.3529s | Speedup=1.94x | Equal=True
Lengthscale   5.0: Fast=0.6834s | Slow=1.3563s | Speedup=1.98x | Equal=True
Lengthscale  10.0: Fast=0.7838s | Slow=1.3482s | Speedup=1.72x | Equal=True
Lengthscale  20.0: Fast=0.7434s | Slow=1.3582s | Speedup=1.83x | Equal=True

HAMMING GRAPH
Lengthscale   1.0: Fast=0.6942s | Slow=1.3436s | Speedup=1.94x | Equal=True
Lengthscale   5.0: Fast=0.8381s | Slow=1.3142s | Speedup=1.57x | Equal=True
Lengthscale  10.0: Fast=0.7756s | Slow=1.3819s | Speedup=1.78x | Equal=True
Lengthscale  20.0: Fast=0.6543s | Slow=1.3453s | Speedup=2.06x | Equal=True

Done!


## Citation

If you are using hypercube or Hamming graphs from GeometricKernels, please consider citing

```
@article{mostowsky2024,
      title = {The GeometricKernels Package: Heat and Matérn Kernels for Geometric Learning on Manifolds, Meshes, and Graphs},
      author = {Peter Mostowsky and Vincent Dutordoir and Iskander Azangulov and Noémie Jaquier and Michael John Hutchinson and Aditya Ravuri and Leonel Rozo and Alexander Terenin and Viacheslav Borovitskiy},
      year = {2024},
      journal = {arXiv:2407.08086},
}
```

```
@inproceedings{doumont2025,
    title = {Omnipresent Yet Overlooked: Heat Kernels in Combinatorial Bayesian Optimization},
    author = {Colin Doumont and Victor Picheny and Viacheslav Borovitskiy and Henry Moss},
    booktitle = {Advances in Neural Information Processing Systems},
    year = {2025},
}
```
